<a href="https://colab.research.google.com/github/Iqra-01/Google_AI_studio_Genai/blob/main/templates/prompting_gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2023 Google LLC

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Setup

In [1]:
!pip install -U -q "google-generativeai>=0.8.2"

In [2]:
# import necessary modules.
import base64
import copy
import json
import pathlib
import requests


import PIL.Image
import IPython.display
from IPython.display import Markdown

try:
    # The SDK will automatically read it from the GOOGLE_API_KEY environment variable.
    # In Colab get the key from Colab-secrets ("🔑" in the left panel).
    import os
    from google.colab import userdata

    os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")
except ImportError:
    pass

import google.generativeai as genai

# Parse the arguments

model = 'gemini-1.5-pro' # @param {isTemplate: true}
contents_b64 = 'W3sicm9sZSI6InVzZXIiLCJwYXJ0cyI6W3sidGV4dCI6ImhvdyBkb2VzIGdlbWluaSB3b3JrXFxuIn1dfSx7InJvbGUiOiJtb2RlbCIsInBhcnRzIjpbeyJ0ZXh0IjoiR2VtaW5pIGlzIEdvb2dsZSdzIG1vc3QgY2FwYWJsZSBtdWx0aW1vZGFsIG1vZGVsLiBJdCdzIGRlc2lnbmVkIHRvIHVuZGVyc3RhbmQgYW5kIGdlbmVyYXRlIHRleHQsIGNvZGUsIGltYWdlcywgYXVkaW8sIGFuZCB2aWRlbyDigJMgYWxsIHdpdGhpbiBhIHNpbmdsZSBtb2RlbC4gIEhlcmUncyBhIGJyZWFrZG93biBvZiBob3cgaXQgd29ya3M6XFxuXFxuKipLZXkgQ29uY2VwdHM6KipcXG5cXG4qICoqTXVsdGltb2RhbDoqKiAgR2VtaW5pJ3MgY29yZSBzdHJlbmd0aCBpcyBpdHMgYWJpbGl0eSB0byBwcm9jZXNzIGFuZCBpbnRlZ3JhdGUgaW5mb3JtYXRpb24gZnJvbSBkaWZmZXJlbnQgbW9kYWxpdGllcyBsaWtlIHRleHQsIGltYWdlcywgYXVkaW8sIGFuZCB2aWRlby4gIFRoaXMgYWxsb3dzIGl0IHRvIHBlcmZvcm0gdGFza3MgdGhhdCByZXF1aXJlIHVuZGVyc3RhbmRpbmcgdGhlIHJlbGF0aW9uc2hpcHMgYmV0d2VlbiB0aGVzZSBtb2RhbGl0aWVzLCBzdWNoIGFzIGRlc2NyaWJpbmcgYW4gaW1hZ2UsIGdlbmVyYXRpbmcgY2FwdGlvbnMgZm9yIHZpZGVvcywgb3IgYW5zd2VyaW5nIHF1ZXN0aW9ucyBhYm91dCBhIGNvbWJpbmF0aW9uIG9mIHRleHQgYW5kIGltYWdlcy5cXG5cXG4qICoqTGFyZ2UgTGFuZ3VhZ2UgTW9kZWwgKExMTSk6KiogQXQgaXRzIGhlYXJ0LCBHZW1pbmkgaXMgYSBsYXJnZSBsYW5ndWFnZSBtb2RlbC4gTExNcyBhcmUgdHJhaW5lZCBvbiBtYXNzaXZlIGRhdGFzZXRzIG9mIHRleHQgYW5kIGNvZGUsIGVuYWJsaW5nIHRoZW0gdG8gbGVhcm4gcGF0dGVybnMsIGdyYW1tYXIsIGFuZCBjb250ZXh0LiBUaGlzIGFsbG93cyB0aGVtIHRvIGdlbmVyYXRlIGh1bWFuLXF1YWxpdHkgdGV4dCwgdHJhbnNsYXRlIGxhbmd1YWdlcywgd3JpdGUgZGlmZmVyZW50IGtpbmRzIG9mIGNyZWF0aXZlIGNvbnRlbnQsIGFuZCBhbnN3ZXIgeW91ciBxdWVzdGlvbnMgaW4gYW4gaW5mb3JtYXRpdmUgd2F5LlxcblxcbiogKipUcmFuc2Zvcm1lciBBcmNoaXRlY3R1cmU6KiogR2VtaW5pIGxpa2VseSB1dGlsaXplcyBhIHRyYW5zZm9ybWVyIGFyY2hpdGVjdHVyZSwgd2hpY2ggaXMgYSBuZXVyYWwgbmV0d29yayBhcmNoaXRlY3R1cmUgZGVzaWduZWQgdG8gaGFuZGxlIHNlcXVlbnRpYWwgZGF0YSBsaWtlIHRleHQgYW5kIGF1ZGlvLiBUcmFuc2Zvcm1lcnMgYXJlIHBhcnRpY3VsYXJseSBnb29kIGF0IGNhcHR1cmluZyBsb25nLXJhbmdlIGRlcGVuZGVuY2llcyBpbiBkYXRhLCB3aGljaCBpcyBjcnVjaWFsIGZvciB1bmRlcnN0YW5kaW5nIGNvbXBsZXggc2VudGVuY2VzIGFuZCBnZW5lcmF0aW5nIGNvaGVyZW50IHJlc3BvbnNlcy5cXG5cXG4qICoqTXVsdGltb2RhbCBUcmFpbmluZzoqKiAgR2VtaW5pJ3MgbXVsdGltb2RhbCBjYXBhYmlsaXRpZXMgY29tZSBmcm9tIGl0cyB0cmFpbmluZyBvbiBkaXZlcnNlIGRhdGFzZXRzIGNvbnRhaW5pbmcgdmFyaW91cyBtb2RhbGl0aWVzLiBUaGlzIHRyYWluaW5nIGxpa2VseSBpbnZvbHZlcyBsZWFybmluZyBqb2ludCByZXByZXNlbnRhdGlvbnMgb2YgZGlmZmVyZW50IG1vZGFsaXRpZXMsIGFsbG93aW5nIHRoZSBtb2RlbCB0byB1bmRlcnN0YW5kIHRoZSByZWxhdGlvbnNoaXBzIGJldHdlZW4gdGhlbS4gIEZvciBpbnN0YW5jZSwgdGhlIG1vZGVsIG1pZ2h0IGxlYXJuIHRoZSBhc3NvY2lhdGlvbiBiZXR3ZWVuIHRoZSBpbWFnZSBvZiBhIGNhdCBhbmQgdGhlIHdvcmQgXFxcImNhdFxcXCIuXFxuXFxuKiAqKkZpbmUtdHVuaW5nOioqICBMaWtlIG90aGVyIGxhcmdlIG1vZGVscywgR2VtaW5pIGNhbiBiZSBmaW5lLXR1bmVkIGZvciBzcGVjaWZpYyB0YXNrcy4gVGhpcyBpbnZvbHZlcyB0cmFpbmluZyB0aGUgbW9kZWwgZnVydGhlciBvbiBhIHNtYWxsZXIsIHRhc2stc3BlY2lmaWMgZGF0YXNldC4gIEZpbmUtdHVuaW5nIGFsbG93cyB0aGUgbW9kZWwgdG8gYWRhcHQgdG8gcGFydGljdWxhciBkb21haW5zIGFuZCBpbXByb3ZlIGl0cyBwZXJmb3JtYW5jZSBvbiB0aG9zZSB0YXNrcy5cXG5cXG5cXG4qKlNpbXBsaWZpZWQgV29ya2Zsb3cgKElsbHVzdHJhdGl2ZSBFeGFtcGxlKToqKlxcblxcbkltYWdpbmUgeW91IGFzayBHZW1pbmksIFxcXCJXaGF0IGJyZWVkIG9mIGRvZyBpcyB0aGlzP1xcXCIgYW5kIHByb3ZpZGUgYSBwaWN0dXJlIG9mIGEgR29sZGVuIFJldHJpZXZlci5cXG5cXG4xLiAqKklucHV0IFByb2Nlc3Npbmc6KiogR2VtaW5pIHByb2Nlc3NlcyB0aGUgaW5wdXQgaW1hZ2UgYW5kIHlvdXIgdGV4dCBxdWVzdGlvbi4gIFRoZSBpbWFnZSBpcyBjb252ZXJ0ZWQgaW50byBhIG51bWVyaWNhbCByZXByZXNlbnRhdGlvbiB0aGF0IHRoZSBtb2RlbCBjYW4gdW5kZXJzdGFuZC5cXG5cXG4yLiAqKk11bHRpbW9kYWwgVW5kZXJzdGFuZGluZzoqKiBHZW1pbmkgY29tYmluZXMgdGhlIGluZm9ybWF0aW9uIGZyb20gdGhlIGltYWdlIGFuZCB0aGUgdGV4dCBxdWVzdGlvbi4gSXRzIG11bHRpbW9kYWwgY2FwYWJpbGl0aWVzIGFsbG93IGl0IHRvIHVuZGVyc3RhbmQgdGhhdCB0aGUgcXVlc3Rpb24gcmVmZXJzIHRvIHRoZSBkb2cgaW4gdGhlIGltYWdlLlxcblxcbjMuICoqSW5mb3JtYXRpb24gUmV0cmlldmFsIGFuZCBSZWFzb25pbmc6KiogR2VtaW5pIHVzZXMgaXRzIGtub3dsZWRnZSBnYWluZWQgZHVyaW5nIHRyYWluaW5nIHRvIGlkZW50aWZ5IHRoZSBicmVlZCBvZiBkb2cgaW4gdGhlIGltYWdlLiAgVGhpcyBpbnZvbHZlcyBhY2Nlc3NpbmcgYW5kIHByb2Nlc3NpbmcgaW5mb3JtYXRpb24gYWJvdXQgZG9nIGJyZWVkcyBhbmQgbWF0Y2hpbmcgaXQgd2l0aCB0aGUgdmlzdWFsIGZlYXR1cmVzIGV4dHJhY3RlZCBmcm9tIHRoZSBpbWFnZS5cXG5cXG40LiAqKk91dHB1dCBHZW5lcmF0aW9uOioqIEdlbWluaSBnZW5lcmF0ZXMgYSB0ZXh0IHJlc3BvbnNlLCBzdWNoIGFzIFxcXCJUaGF0J3MgYSBHb2xkZW4gUmV0cmlldmVyLlxcXCJcXG5cXG5cXG4qKktleSBEaWZmZXJlbmNlcyBmcm9tIE90aGVyIE1vZGVsczoqKlxcblxcbkdlbWluaSBzdGFuZHMgb3V0IGJlY2F1c2Ugb2YgaXRzIG5hdGl2ZSBtdWx0aW1vZGFsIGNhcGFiaWxpdGllcy4gVW5saWtlIHNvbWUgbW9kZWxzIHRoYXQgaGFuZGxlIGRpZmZlcmVudCBtb2RhbGl0aWVzIHNlcGFyYXRlbHksIEdlbWluaSBpcyBkZXNpZ25lZCB0byBwcm9jZXNzIGFuZCBpbnRlZ3JhdGUgdGhlbSBzZWFtbGVzc2x5LiBUaGlzIGFsbG93cyBmb3IgbW9yZSBzb3BoaXN0aWNhdGVkIGFwcGxpY2F0aW9ucyBhbmQgYSBkZWVwZXIgdW5kZXJzdGFuZGluZyBvZiB0aGUgcmVsYXRpb25zaGlwcyBiZXR3ZWVuIGRpZmZlcmVudCBmb3JtcyBvZiBpbmZvcm1hdGlvbi5cXG5cXG4qKkxpbWl0YXRpb25zOioqXFxuXFxuTGlrZSBhbGwgQUkgbW9kZWxzLCBHZW1pbmkgaGFzIGxpbWl0YXRpb25zLiBJdCBjYW4gc29tZXRpbWVzIGdlbmVyYXRlIGluY29ycmVjdCBvciBiaWFzZWQgaW5mb3JtYXRpb24sIGhhbGx1Y2luYXRlIGRldGFpbHMsIG9yIHN0cnVnZ2xlIHdpdGggbnVhbmNlZCByZWFzb25pbmcuICBPbmdvaW5nIHJlc2VhcmNoIGFuZCBkZXZlbG9wbWVudCBhcmUgYWltZWQgYXQgYWRkcmVzc2luZyB0aGVzZSBsaW1pdGF0aW9ucy5cXG4ifV19LHsicm9sZSI6InVzZXIiLCJwYXJ0cyI6W3sidGV4dCI6InRlbGwgbWUgbW9yZSBhYm91dCBsbG1zXFxuIn1dfSx7InJvbGUiOiJtb2RlbCIsInBhcnRzIjpbeyJ0ZXh0IjoiTGFyZ2UgTGFuZ3VhZ2UgTW9kZWxzIChMTE1zKSBhcmUgYSB0eXBlIG9mIGFydGlmaWNpYWwgaW50ZWxsaWdlbmNlIChBSSkgdGhhdCBhcmUgZGVzaWduZWQgdG8gdW5kZXJzdGFuZCBhbmQgZ2VuZXJhdGUgaHVtYW4gbGFuZ3VhZ2UuIFRoZXkgYXJlIHRyYWluZWQgb24gbWFzc2l2ZSBhbW91bnRzIG9mIHRleHQgZGF0YSwgYWxsb3dpbmcgdGhlbSB0byBsZWFybiBwYXR0ZXJucywgZ3JhbW1hciwgc2VtYW50aWNzLCBhbmQgZXZlbiBzb21lIHdvcmxkIGtub3dsZWRnZS5cXG5cXG5IZXJlJ3MgYSBicmVha2Rvd24gb2Yga2V5IGFzcGVjdHMgb2YgTExNczpcXG5cXG4qKjEuIFRyYWluaW5nIERhdGEgYW5kIFByb2Nlc3M6KipcXG5cXG4qICoqTWFzc2l2ZSBEYXRhc2V0czoqKiBMTE1zIGFyZSB0cmFpbmVkIG9uIGluY3JlZGlibHkgbGFyZ2UgZGF0YXNldHMgb2YgdGV4dCBhbmQgY29kZSwgb2Z0ZW4gcGV0YWJ5dGVzIGluIHNpemUuIFRoaXMgZGF0YSBjYW4gY29tZSBmcm9tIHZhcmlvdXMgc291cmNlcyBsaWtlIGJvb2tzLCBhcnRpY2xlcywgd2Vic2l0ZXMsIGNvZGUgcmVwb3NpdG9yaWVzLCBhbmQgbW9yZS5cXG4qICoqU2VsZi1TdXBlcnZpc2VkIExlYXJuaW5nOioqICBBIGNvbW1vbiB0cmFpbmluZyBtZXRob2QgaXMgc2VsZi1zdXBlcnZpc2VkIGxlYXJuaW5nLCB3aGVyZSB0aGUgbW9kZWwgbGVhcm5zIGJ5IHByZWRpY3RpbmcgbWFza2VkIHdvcmRzIG9yIHBocmFzZXMgd2l0aGluIGEgc2VudGVuY2UuIFRoaXMgYWxsb3dzIHRoZSBtb2RlbCB0byBsZWFybiB0aGUgcmVsYXRpb25zaGlwcyBiZXR3ZWVuIHdvcmRzIGFuZCBkZXZlbG9wIGEgcmljaCB1bmRlcnN0YW5kaW5nIG9mIGxhbmd1YWdlLlxcbiogKipUcmFuc2Zvcm1lciBBcmNoaXRlY3R1cmU6KiogTW9zdCBtb2Rlcm4gTExNcyBhcmUgYmFzZWQgb24gdGhlIHRyYW5zZm9ybWVyIGFyY2hpdGVjdHVyZSwgYSBuZXVyYWwgbmV0d29yayBhcmNoaXRlY3R1cmUgcGFydGljdWxhcmx5IGVmZmVjdGl2ZSBhdCBwcm9jZXNzaW5nIHNlcXVlbnRpYWwgZGF0YSBsaWtlIHRleHQuIFRyYW5zZm9ybWVycyB1dGlsaXplIGEgbWVjaGFuaXNtIGNhbGxlZCBcXFwiYXR0ZW50aW9uXFxcIiB3aGljaCBhbGxvd3MgdGhlIG1vZGVsIHRvIHdlaWdoIHRoZSBpbXBvcnRhbmNlIG9mIGRpZmZlcmVudCB3b3JkcyBpbiBhIHNlbnRlbmNlIHdoZW4gZ2VuZXJhdGluZyBhIHJlc3BvbnNlLlxcblxcbioqMi4gQ2FwYWJpbGl0aWVzOioqXFxuXFxuKiAqKlRleHQgR2VuZXJhdGlvbjoqKiBMTE1zIGNhbiBnZW5lcmF0ZSB2YXJpb3VzIHR5cGVzIG9mIHRleHQsIGluY2x1ZGluZyBzdG9yaWVzLCBwb2VtcywgYXJ0aWNsZXMsIHN1bW1hcmllcywgYW5kIGNvZGUuXFxuKiAqKlRyYW5zbGF0aW9uOioqICBUaGV5IGNhbiB0cmFuc2xhdGUgYmV0d2VlbiBtdWx0aXBsZSBsYW5ndWFnZXMuXFxuKiAqKlF1ZXN0aW9uIEFuc3dlcmluZzoqKiBUaGV5IGNhbiBhbnN3ZXIgcXVlc3Rpb25zIGJhc2VkIG9uIHRoZSBpbmZvcm1hdGlvbiB0aGV5IGhhdmUgbGVhcm5lZCBmcm9tIHRoZWlyIHRyYWluaW5nIGRhdGEuXFxuKiAqKkRpYWxvZ3VlOioqIFRoZXkgY2FuIGVuZ2FnZSBpbiBjb252ZXJzYXRpb25zIGFuZCBnZW5lcmF0ZSBjb250ZXh0dWFsbHkgcmVsZXZhbnQgcmVzcG9uc2VzLlxcbiogKipDb2RlIEdlbmVyYXRpb246KiogIExMTXMgY2FuIGdlbmVyYXRlIGNvZGUgaW4gdmFyaW91cyBwcm9ncmFtbWluZyBsYW5ndWFnZXMgYW5kIGV2ZW4gZXhwbGFpbiBleGlzdGluZyBjb2RlLlxcbiogKipTdW1tYXJpemF0aW9uOioqICBUaGV5IGNhbiBjb25kZW5zZSBsb25nZXIgdGV4dHMgaW50byBjb25jaXNlIHN1bW1hcmllcy5cXG5cXG4qKjMuIEtleSBDb25jZXB0czoqKlxcblxcbiogKipDb250ZXh0IFdpbmRvdzoqKiBMTE1zIGhhdmUgYSBsaW1pdGVkIFxcXCJjb250ZXh0IHdpbmRvdyxcXFwiIG1lYW5pbmcgdGhleSBjYW4gb25seSBjb25zaWRlciBhIGNlcnRhaW4gYW1vdW50IG9mIHRleHQgYXQgYSB0aW1lLiAgQW55dGhpbmcgYmV5b25kIHRoaXMgd2luZG93IGlzIG5vdCBkaXJlY3RseSBjb25zaWRlcmVkIHdoZW4gZ2VuZXJhdGluZyBhIHJlc3BvbnNlLlxcbiogKipUb2tlbml6YXRpb246KiogSW5wdXQgdGV4dCBpcyBicm9rZW4gZG93biBpbnRvIHNtYWxsZXIgdW5pdHMgY2FsbGVkIFxcXCJ0b2tlbnMsXFxcIiB3aGljaCBjYW4gYmUgd29yZHMsIHN1YndvcmRzLCBvciBldmVuIGNoYXJhY3RlcnMuIFRoaXMgYWxsb3dzIHRoZSBtb2RlbCB0byBwcm9jZXNzIGFuZCByZXByZXNlbnQgdGhlIHRleHQgbnVtZXJpY2FsbHkuXFxuKiAqKkVtYmVkZGluZ3M6KiogV29yZHMgYW5kIHBocmFzZXMgYXJlIHJlcHJlc2VudGVkIGFzIG51bWVyaWNhbCB2ZWN0b3JzIGNhbGxlZCBcXFwiZW1iZWRkaW5ncyxcXFwiIGNhcHR1cmluZyB0aGVpciBzZW1hbnRpYyBtZWFuaW5nIGFuZCByZWxhdGlvbnNoaXBzIHdpdGggb3RoZXIgd29yZHMuXFxuKiAqKlByb21wdCBFbmdpbmVlcmluZzoqKiAgVGhlIHByb2Nlc3Mgb2YgY3JhZnRpbmcgZWZmZWN0aXZlIGlucHV0IHByb21wdHMgdG8gZ3VpZGUgdGhlIExMTSB0b3dhcmRzIGRlc2lyZWQgb3V0cHV0cy4gIEEgd2VsbC1jcmFmdGVkIHByb21wdCBjYW4gc2lnbmlmaWNhbnRseSBpbXByb3ZlIHRoZSBxdWFsaXR5IGFuZCByZWxldmFuY2Ugb2YgdGhlIGdlbmVyYXRlZCB0ZXh0LlxcblxcbioqNC4gRXhhbXBsZXMgb2YgTExNczoqKlxcblxcbiogKipHUFQgKEdlbmVyYXRpdmUgUHJlLXRyYWluZWQgVHJhbnNmb3JtZXIpIG1vZGVscyBieSBPcGVuQUk6KiogIEEgcG9wdWxhciBzZXJpZXMgb2YgTExNcyBrbm93biBmb3IgdGhlaXIgcG93ZXJmdWwgdGV4dCBnZW5lcmF0aW9uIGNhcGFiaWxpdGllcy5cXG4qICoqTGFNREEgKExhbmd1YWdlIE1vZGVsIGZvciBEaWFsb2d1ZSBBcHBsaWNhdGlvbnMpIGJ5IEdvb2dsZToqKiAgRm9jdXNlZCBvbiBjb252ZXJzYXRpb25hbCBBSSBhbmQgZ2VuZXJhdGluZyBuYXR1cmFsIGFuZCBlbmdhZ2luZyBkaWFsb2d1ZS5cXG4qICoqUGFMTSAoUGF0aHdheXMgTGFuZ3VhZ2UgTW9kZWwpIGJ5IEdvb2dsZToqKiBBIGxhcmdlIGxhbmd1YWdlIG1vZGVsIGNhcGFibGUgb2YgcGVyZm9ybWluZyBhIHdpZGUgcmFuZ2Ugb2YgdGFza3MsIGluY2x1ZGluZyBsYW5ndWFnZSB1bmRlcnN0YW5kaW5nLCByZWFzb25pbmcsIGFuZCBjb2RlIGdlbmVyYXRpb24uXFxuKiAqKkxMYU1BIChMYXJnZSBMYW5ndWFnZSBNb2RlbCBNZXRhIEFJKSBieSBNZXRhOioqICBBbiBvcGVuLXNvdXJjZSBMTE0gZGVzaWduZWQgdG8gYmUgbW9yZSBhY2Nlc3NpYmxlIHRvIHJlc2VhcmNoZXJzLlxcblxcbioqNS4gTGltaXRhdGlvbnM6KipcXG5cXG4qICoqQmlhczoqKiBMTE1zIGNhbiBpbmhlcml0IGJpYXNlcyBwcmVzZW50IGluIHRoZWlyIHRyYWluaW5nIGRhdGEsIHBvdGVudGlhbGx5IGxlYWRpbmcgdG8gdW5mYWlyIG9yIGRpc2NyaW1pbmF0b3J5IG91dHB1dHMuXFxuKiAqKkhhbGx1Y2luYXRpb25zOioqIFRoZXkgY2FuIHNvbWV0aW1lcyBnZW5lcmF0ZSBmYWN0dWFsbHkgaW5jb3JyZWN0IG9yIG5vbnNlbnNpY2FsIGluZm9ybWF0aW9uLCBvZnRlbiByZWZlcnJlZCB0byBhcyBcXFwiaGFsbHVjaW5hdGlvbnMuXFxcIlxcbiogKipMYWNrIG9mIFJlYWwtV29ybGQgVW5kZXJzdGFuZGluZzoqKiAgV2hpbGUgTExNcyBjYW4gbGVhcm4gYSBsb3QgZnJvbSB0ZXh0LCB0aGV5IGxhY2sgdHJ1ZSB1bmRlcnN0YW5kaW5nIG9mIHRoZSByZWFsIHdvcmxkIGFuZCBjYW4gc3RydWdnbGUgd2l0aCBjb21tb24gc2Vuc2UgcmVhc29uaW5nLlxcbiogKipFdGhpY2FsIENvbmNlcm5zOioqICBUaGUgcG90ZW50aWFsIG1pc3VzZSBvZiBMTE1zIGZvciBnZW5lcmF0aW5nIG1pc2luZm9ybWF0aW9uLCBzcGFtLCBvciBoYXJtZnVsIGNvbnRlbnQgcmFpc2VzIGV0aGljYWwgY29uY2VybnMuXFxuXFxuXFxuTExNcyBhcmUgY29uc3RhbnRseSBldm9sdmluZywgYW5kIG9uZ29pbmcgcmVzZWFyY2ggYWltcyB0byBhZGRyZXNzIHRoZWlyIGxpbWl0YXRpb25zIGFuZCBpbXByb3ZlIHRoZWlyIGNhcGFiaWxpdGllcy4gVGhleSByZXByZXNlbnQgYSBzaWduaWZpY2FudCBhZHZhbmNlbWVudCBpbiBBSSBhbmQgaGF2ZSB0aGUgcG90ZW50aWFsIHRvIHRyYW5zZm9ybSB2YXJpb3VzIGZpZWxkcywgZnJvbSBlZHVjYXRpb24gYW5kIGhlYWx0aGNhcmUgdG8gZW50ZXJ0YWlubWVudCBhbmQgc29mdHdhcmUgZGV2ZWxvcG1lbnQuXFxuIn1dfV0=' # @param {isTemplate: true}
generation_config_b64 = 'eyJ0ZW1wZXJhdHVyZSI6MSwidG9wX3AiOjAuOTUsInRvcF9rIjo0MCwibWF4X291dHB1dF90b2tlbnMiOjgxOTJ9' # @param {isTemplate: true}
safety_settings_b64 = "e30="  # @param {isTemplate: true}

gais_contents = json.loads(base64.b64decode(contents_b64))

generation_config = json.loads(base64.b64decode(generation_config_b64))
safety_settings = json.loads(base64.b64decode(safety_settings_b64))

stream = False

# Convert and upload the files

tempfiles = pathlib.Path(f"tempfiles")
tempfiles.mkdir(parents=True, exist_ok=True)


drive = None
def upload_file_data(file_data, index):
    """Upload files to the Files API.

    For each file, Google AI Studio either sent:
    - a Google Drive ID,
    - a URL,
    - a file path, or
    - The raw bytes (`inline_data`).

    The API only understands `inline_data` or it's Files API.
    This code, uploads files to the files API where the API can access them.
    """

    mime_type = file_data["mime_type"]
    if drive_id := file_data.pop("drive_id", None):
        if drive is None:
          from google.colab import drive
          drive.mount("/gdrive")

        path = next(
            pathlib.Path(f"/gdrive/.shortcut-targets-by-id/{drive_id}").glob("*")
        )
        print("Uploading:", str(path))
        file_info = genai.upload_file(path=path, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if url := file_data.pop("url", None):
        response = requests.get(url)
        data = response.content
        name = url.split("/")[-1]
        path = tempfiles / str(index)
        path.write_bytes(data)
        print("Uploading:", url)
        file_info = genai.upload_file(path, display_name=name, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if name := file_data.get("filename", None):
        if not pathlib.Path(name).exists():
            raise IOError(
                f"local file: `{name}` does not exist. You can upload files "
                'to Colab using the file manager ("📁 Files" in the left '
                "toolbar)"
            )
        file_info = genai.upload_file(path, display_name=name, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if "inline_data" in file_data:
        return

    raise ValueError("Either `drive_id`, `url` or `inline_data` must be provided.")


contents = copy.deepcopy(gais_contents)

index = 0
for content in contents:
    for n, part in enumerate(content["parts"]):
        if file_data := part.get("file_data", None):
            upload_file_data(file_data, index)
            index += 1

import json
print(json.dumps(contents, indent=4))

[
    {
        "role": "user",
        "parts": [
            {
                "text": "how does gemini work\\n"
            }
        ]
    },
    {
        "role": "model",
        "parts": [
            {
                "text": "Gemini is Google's most capable multimodal model. It's designed to understand and generate text, code, images, audio, and video \u2013 all within a single model.  Here's a breakdown of how it works:\\n\\n**Key Concepts:**\\n\\n* **Multimodal:**  Gemini's core strength is its ability to process and integrate information from different modalities like text, images, audio, and video.  This allows it to perform tasks that require understanding the relationships between these modalities, such as describing an image, generating captions for videos, or answering questions about a combination of text and images.\\n\\n* **Large Language Model (LLM):** At its heart, Gemini is a large language model. LLMs are trained on massive datasets of text and code, enabling th

## Call `generate_content`

In [4]:
from IPython.display import display
from IPython.display import Markdown

# Call the model and print the response.
gemini = genai.GenerativeModel(model_name=model)

response = gemini.generate_content(
    contents,
    generation_config=generation_config,
    safety_settings=safety_settings,
    stream=stream,
)

if response.candidates:
    display(Markdown(response.text))
else:
    # Print prompt feedback if available to understand the blocking reason
    if response.prompt_feedback:
        print(f"Prompt feedback: {response.prompt_feedback}")
    else:
        print("Error: No candidates returned. The prompt may have been blocked.")

Error: No candidates returned. The prompt may have been blocked.


<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://ai.google.dev/gemini-api/docs"><img src="https://ai.google.dev/static/site-assets/images/docs/notebook-site-button.png" height="32" width="32" />Docs on ai.google.dev</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/google-gemini/cookbook/blob/main/quickstarts"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />More notebooks in the Cookbook</a>
  </td>
</table>

## [optional] Show the conversation

This section displays the conversation received from Google AI Studio.

In [5]:
# @title Show the conversation, in colab.
import mimetypes

def show_file(file_data):
    mime_type = file_data["mime_type"]

    if drive_id := file_data.get("drive_id", None):
        path = next(
            pathlib.Path(f"/gdrive/.shortcut-targets-by-id/{drive_id}").glob("*")
        )
        name = path
        # data = path.read_bytes()
        kwargs = {"filename": path}
    elif url := file_data.get("url", None):
        name = url
        kwargs = {"url": url}
        # response = requests.get(url)
        # data = response.content
    elif data := file_data.get("inline_data", None):
        name = None
        kwargs = {"data": data}
    elif name := file_data.get("filename", None):
        if not pathlib.Path(name).exists():
            raise IOError(
                f"local file: `{name}` does not exist. You can upload files to "
                'Colab using the file manager ("📁 Files"in the left toolbar)'
            )
    else:
        raise ValueError("Either `drive_id`, `url` or `inline_data` must be provided.")

        print(f"File:\n    name: {name}\n    mime_type: {mime_type}\n")
        return

    format = mimetypes.guess_extension(mime_type).strip(".")
    if mime_type.startswith("image/"):
        image = IPython.display.Image(**kwargs, width=256)
        IPython.display.display(image)
        print()
        return

    if mime_type.startswith("audio/"):
        if len(data) < 2**12:
            audio = IPython.display.Audio(**kwargs)
            IPython.display.display(audio)
            print()
            return

    if mime_type.startswith("video/"):
        if len(data) < 2**12:
            audio = IPython.display.Video(**kwargs, mimetype=mime_type)
            IPython.display.display(audio)
            print()
            return

    print(f"File:\n    name: {name}\n    mime_type: {mime_type}\n")


for content in gais_contents:
    if role := content.get("role", None):
        print("Role:", role, "\n")

    for n, part in enumerate(content["parts"]):
        if text := part.get("text", None):
            print(text, "\n")

        elif file_data := part.get("file_data", None):
            show_file(file_data)

    print("-" * 80, "\n")

Role: user 

how does gemini work\n 

-------------------------------------------------------------------------------- 

Role: model 

Gemini is Google's most capable multimodal model. It's designed to understand and generate text, code, images, audio, and video – all within a single model.  Here's a breakdown of how it works:\n\n**Key Concepts:**\n\n* **Multimodal:**  Gemini's core strength is its ability to process and integrate information from different modalities like text, images, audio, and video.  This allows it to perform tasks that require understanding the relationships between these modalities, such as describing an image, generating captions for videos, or answering questions about a combination of text and images.\n\n* **Large Language Model (LLM):** At its heart, Gemini is a large language model. LLMs are trained on massive datasets of text and code, enabling them to learn patterns, grammar, and context. This allows them to generate human-quality text, translate language